# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [188]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [189]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [190]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [191]:
df = data[["CustomerID", "ProductName", "Quantity"]]
df = df.groupby(["CustomerID", "ProductName"]).agg([sum])
df.reset_index(inplace=True)
df.columns = df.columns.get_level_values(0)
df

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [192]:
df = df.pivot(index="ProductName", columns="CustomerID", values="Quantity")
df = df.fillna(0)
df

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [193]:
distances = pd.DataFrame(1/(1 + squareform(pdist(df.T, 'euclidean'))), 
                         index=df.columns, columns=df.columns)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [195]:
columns = [str(e).strip() for e in list(distances.columns)]
distances.columns = columns
print(distances["33"].sort_values(ascending=False).head(6))
top_selection = list(distances["33"].sort_values(ascending=False).head(6).index[1:])
top_selection

CustomerID
33      1.000000
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64


[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [118]:
df = data[["CustomerID", "ProductName", "Quantity"]]
df = df.groupby(["CustomerID", "ProductName"]).agg([sum])
df.reset_index(inplace=True)
df.columns = df.columns.get_level_values(0)
df

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


In [136]:
df_filter = df[df.CustomerID.isin(top_selection)]
df_filter

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1
...,...,...,...
3003,3535,Tray - 16in Rnd Blk,1
3004,3535,Wanton Wrap,1
3005,3535,Wine - Blue Nun Qualitatswein,1
3006,3535,"Wine - Magnotta, Merlot Sr Vqa",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [145]:
df_filter = df_filter[["ProductName", "Quantity"]]
df_filter = df_filter.groupby(["ProductName"]).agg([sum])
# df_filter.sort_values(ascending=False, by=["Quantity"], inplace=True)
df_filter.reset_index(inplace=True)
df_filter.columns = df_filter.columns.get_level_values(0)
df_filter = df_filter.sort_values(by=["Quantity"], ascending=False)
df_filter

,ProductName,Quantity
33,Butter - Unsalted,3
203,Wine - Ej Gallo Sierra Valley,3
186,Towels - Paper / Kraft,3
168,Soup - Campbells Bean Medley,3
199,Wine - Blue Nun Qualitatswein,3
...,...,...
85,Hinge W Undercut,1
86,Ice Cream Bar - Hageen Daz To,1
87,Jagermeister,1
88,Jolt Cola - Electric Blue,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [171]:
top_products = df_filter.ProductName.unique()
top5_not_purchased = df[(df.CustomerID != 33) & 
                        (df.ProductName.isin(top_products))].sort_values(by=["Quantity"], ascending=False)
top5_not_purchased.head(5)

,CustomerID,ProductName,Quantity
62492,97029,Fenngreek Seed,75
63267,97900,Scampi Tail,75
59324,91443,Scallops - 10/20,72
60119,92525,Wine - Chablis 2003 Champs,72
60309,92673,Anchovy Paste - 56 G Tube,72


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [211]:
df = data[["CustomerID", "ProductName", "Quantity"]]
df = df.groupby(["CustomerID", "ProductName"]).agg([sum])
df.reset_index(inplace=True)
df.columns = df.columns.get_level_values(0)
df.CustomerID = df.CustomerID.astype(str)
type(df.CustomerID.iloc[0])

str

In [217]:
df_pivot = df.pivot(index="ProductName", columns="CustomerID", values="Quantity")
df_pivot = df_pivot.fillna(0)
df_pivot

CustomerID,10016,1008,10094,10114,10224,10311,1034,10418,1066,1072,...,97753,97769,97793,97900,97928,9804,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,25.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,25.0,25.0,0.0,25.0,0.0,25.0,0.0
Apricots - Dried,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,...,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,1.0,0.0,...,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [218]:
distances = pd.DataFrame(1/(1 + squareform(pdist(df_pivot.T, 'euclidean'))), 
                         index=df_pivot.columns, columns=df_pivot.columns)

In [219]:
dictionary = {}
unique_customers = df.CustomerID.unique()


for e in unique_customers:
    top_selection = list(distances[e].sort_values(ascending=False).head(6).index[1:])
    df_filter = df[df.CustomerID.isin(top_selection)]
    df_filter = df_filter[["ProductName", "Quantity"]]
    df_filter = df_filter.groupby(["ProductName"]).agg([sum])
    df_filter.reset_index(inplace=True)
    df_filter.columns = df_filter.columns.get_level_values(0)
    df_filter = df_filter.sort_values(by=["Quantity"], ascending=False)
    top_products = df_filter.ProductName.unique()
    top5_not_purchased = df[(df.CustomerID != e) & 
                        (df.ProductName.isin(top_products))].sort_values(by=["Quantity"], ascending=False)
    dictionary[e] = list(top5_not_purchased.head(5).ProductName)

dictionary



{'33': ['Fenngreek Seed',
  'Scampi Tail',
  'Scallops - 10/20',
  'Wine - Chablis 2003 Champs',
  'Anchovy Paste - 56 G Tube'],
 '200': ['Longos - Grilled Salmon With Bbq',
  'Yeast Dry - Fermipan',
  'Scampi Tail',
  'Coffee - Irish Cream',
  'Fenngreek Seed'],
 '264': ['Yeast Dry - Fermipan',
  'Veal - Eye Of Round',
  'Coffee - Irish Cream',
  'Fenngreek Seed',
  'Scallops - 10/20'],
 '356': ['Spice - Peppercorn Melange',
  'Fenngreek Seed',
  'Scampi Tail',
  'Crab - Dungeness, Whole',
  'Wine - Two Oceans Cabernet'],
 '412': ['Yeast Dry - Fermipan',
  'Fenngreek Seed',
  'Coffee - Irish Cream',
  'Scampi Tail',
  'Crab - Dungeness, Whole'],
 '464': ['Scampi Tail',
  'Fenngreek Seed',
  'Anchovy Paste - 56 G Tube',
  'Scallops - 10/20',
  'Wine - Chablis 2003 Champs'],
 '477': ['Yeast Dry - Fermipan',
  'Fenngreek Seed',
  'Coffee - Irish Cream',
  'Scampi Tail',
  'Crab - Dungeness, Whole'],
 '639': ['Longos - Grilled Salmon With Bbq',
  'Coffee - Irish Cream',
  'Scampi Tail',
 

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [221]:
results = pd.DataFrame(dictionary)
results

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Fenngreek Seed,Longos - Grilled Salmon With Bbq,Yeast Dry - Fermipan,Spice - Peppercorn Melange,Yeast Dry - Fermipan,Scampi Tail,Yeast Dry - Fermipan,Longos - Grilled Salmon With Bbq,Fenngreek Seed,Fenngreek Seed,...,Longos - Grilled Salmon With Bbq,Yeast Dry - Fermipan,Longos - Grilled Salmon With Bbq,Yeast Dry - Fermipan,Longos - Grilled Salmon With Bbq,Longos - Grilled Salmon With Bbq,Longos - Grilled Salmon With Bbq,Bread - French Baquette,Longos - Grilled Salmon With Bbq,Yeast Dry - Fermipan
1,Scampi Tail,Yeast Dry - Fermipan,Veal - Eye Of Round,Fenngreek Seed,Fenngreek Seed,Fenngreek Seed,Fenngreek Seed,Coffee - Irish Cream,Scampi Tail,Scampi Tail,...,Yeast Dry - Fermipan,Fenngreek Seed,Bread - French Baquette,Fenngreek Seed,Yeast Dry - Fermipan,Yeast Dry - Fermipan,Bread - French Baquette,Coffee - Irish Cream,Yeast Dry - Fermipan,Fenngreek Seed
2,Scallops - 10/20,Scampi Tail,Coffee - Irish Cream,Scampi Tail,Coffee - Irish Cream,Anchovy Paste - 56 G Tube,Coffee - Irish Cream,Scampi Tail,Wine - Chablis 2003 Champs,Anchovy Paste - 56 G Tube,...,Fenngreek Seed,Coffee - Irish Cream,Spice - Peppercorn Melange,Coffee - Irish Cream,Bread - French Baquette,Spice - Peppercorn Melange,Fenngreek Seed,Spice - Peppercorn Melange,Spice - Peppercorn Melange,Veal - Eye Of Round
3,Wine - Chablis 2003 Champs,Coffee - Irish Cream,Fenngreek Seed,"Crab - Dungeness, Whole",Scampi Tail,Scallops - 10/20,Scampi Tail,Fenngreek Seed,Anchovy Paste - 56 G Tube,Scallops - 10/20,...,Veal - Eye Of Round,Bread - French Baquette,Scampi Tail,Bread - French Baquette,Spice - Peppercorn Melange,Veal - Eye Of Round,Spice - Peppercorn Melange,Veal - Eye Of Round,Bread - French Baquette,Scampi Tail
4,Anchovy Paste - 56 G Tube,Fenngreek Seed,Scallops - 10/20,Wine - Two Oceans Cabernet,"Crab - Dungeness, Whole",Wine - Chablis 2003 Champs,"Crab - Dungeness, Whole",Garlic - Peeled,"Crab - Dungeness, Whole","Crab - Dungeness, Whole",...,Scallops - 10/20,Anchovy Paste - 56 G Tube,Fenngreek Seed,Anchovy Paste - 56 G Tube,Garlic - Peeled,Bread - French Baquette,Scampi Tail,Fenngreek Seed,Scampi Tail,Coffee - Irish Cream


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [222]:
df = data[["CustomerID", "ProductName", "Quantity"]]
df = df.groupby(["CustomerID", "ProductName"]).agg([sum])
df.reset_index(inplace=True)
df.columns = df.columns.get_level_values(0)
df.CustomerID = df.CustomerID.astype(str)
type(df.CustomerID.iloc[0])

str

In [223]:
df_pivot = df.pivot(index="ProductName", columns="CustomerID", values="Quantity")
df_pivot = df_pivot.fillna(0)
df_pivot

CustomerID,10016,1008,10094,10114,10224,10311,1034,10418,1066,1072,...,97753,97769,97793,97900,97928,9804,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,25.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,25.0,25.0,0.0,25.0,0.0,25.0,0.0
Apricots - Dried,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,...,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,1.0,0.0,...,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [225]:
distances = pd.DataFrame(1/(1 + squareform(pdist(df_pivot.T, 'correlation'))), 
                         index=df_pivot.columns, columns=df_pivot.columns)

In [226]:
dictionary = {}
unique_customers = df.CustomerID.unique()


for e in unique_customers:
    top_selection = list(distances[e].sort_values(ascending=False).head(6).index[1:])
    df_filter = df[df.CustomerID.isin(top_selection)]
    df_filter = df_filter[["ProductName", "Quantity"]]
    df_filter = df_filter.groupby(["ProductName"]).agg([sum])
    df_filter.reset_index(inplace=True)
    df_filter.columns = df_filter.columns.get_level_values(0)
    df_filter = df_filter.sort_values(by=["Quantity"], ascending=False)
    top_products = df_filter.ProductName.unique()
    top5_not_purchased = df[(df.CustomerID != e) & 
                        (df.ProductName.isin(top_products))].sort_values(by=["Quantity"], ascending=False)
    dictionary[e] = list(top5_not_purchased.head(5).ProductName)

dictionary


{'33': ['Longos - Grilled Salmon With Bbq',
  'Yeast Dry - Fermipan',
  'Scampi Tail',
  'Bread - French Baquette',
  'Fenngreek Seed'],
 '200': ['Veal - Eye Of Round',
  'Scampi Tail',
  'Fenngreek Seed',
  'Anchovy Paste - 56 G Tube',
  'Wine - Chablis 2003 Champs'],
 '264': ['Yeast Dry - Fermipan',
  'Fenngreek Seed',
  'Bread - French Baquette',
  'Appetizer - Mini Egg Roll, Shrimp',
  'Wine - Chablis 2003 Champs'],
 '356': ['Longos - Grilled Salmon With Bbq',
  'Veal - Eye Of Round',
  'Spice - Peppercorn Melange',
  'Scallops - 10/20',
  'Wine - Two Oceans Cabernet'],
 '412': ['Coffee - Irish Cream',
  'Spice - Peppercorn Melange',
  'Scampi Tail',
  'Veal - Eye Of Round',
  'Crab - Dungeness, Whole'],
 '464': ['Yeast Dry - Fermipan',
  'Wine - Two Oceans Cabernet',
  'Scallops - 10/20',
  'Anchovy Paste - 56 G Tube',
  'Crab - Dungeness, Whole'],
 '477': ['Bread - French Baquette',
  'Scampi Tail',
  'Veal - Eye Of Round',
  'Anchovy Paste - 56 G Tube',
  'Crab - Dungeness, Whol

In [227]:
results = pd.DataFrame(dictionary)
results

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Longos - Grilled Salmon With Bbq,Veal - Eye Of Round,Yeast Dry - Fermipan,Longos - Grilled Salmon With Bbq,Coffee - Irish Cream,Yeast Dry - Fermipan,Bread - French Baquette,Yeast Dry - Fermipan,Coffee - Irish Cream,Yeast Dry - Fermipan,...,Longos - Grilled Salmon With Bbq,Longos - Grilled Salmon With Bbq,Longos - Grilled Salmon With Bbq,Coffee - Irish Cream,Coffee - Irish Cream,Longos - Grilled Salmon With Bbq,Longos - Grilled Salmon With Bbq,Longos - Grilled Salmon With Bbq,Longos - Grilled Salmon With Bbq,Yeast Dry - Fermipan
1,Yeast Dry - Fermipan,Scampi Tail,Fenngreek Seed,Veal - Eye Of Round,Spice - Peppercorn Melange,Wine - Two Oceans Cabernet,Scampi Tail,Fenngreek Seed,Spice - Peppercorn Melange,Scampi Tail,...,Fenngreek Seed,Yeast Dry - Fermipan,Scampi Tail,Veal - Eye Of Round,Bread - French Baquette,Yeast Dry - Fermipan,Spice - Peppercorn Melange,Fenngreek Seed,Yeast Dry - Fermipan,Veal - Eye Of Round
2,Scampi Tail,Fenngreek Seed,Bread - French Baquette,Spice - Peppercorn Melange,Scampi Tail,Scallops - 10/20,Veal - Eye Of Round,Spice - Peppercorn Melange,Veal - Eye Of Round,Bread - French Baquette,...,"Appetizer - Mini Egg Roll, Shrimp",Veal - Eye Of Round,Spice - Peppercorn Melange,Fenngreek Seed,Spice - Peppercorn Melange,Scampi Tail,Coffee - Irish Cream,Scampi Tail,Spice - Peppercorn Melange,Fenngreek Seed
3,Bread - French Baquette,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Scallops - 10/20,Veal - Eye Of Round,Anchovy Paste - 56 G Tube,Anchovy Paste - 56 G Tube,Veal - Eye Of Round,Fenngreek Seed,Wine - Chablis 2003 Champs,...,Scallops - 10/20,Fenngreek Seed,Fenngreek Seed,Garlic - Peeled,Scallops - 10/20,Coffee - Irish Cream,Fenngreek Seed,"Crab - Dungeness, Whole",Scampi Tail,"Appetizer - Mini Egg Roll, Shrimp"
4,Fenngreek Seed,Wine - Chablis 2003 Champs,Wine - Chablis 2003 Champs,Wine - Two Oceans Cabernet,"Crab - Dungeness, Whole","Crab - Dungeness, Whole","Crab - Dungeness, Whole",Wine - Two Oceans Cabernet,Garlic - Peeled,"Appetizer - Mini Egg Roll, Shrimp",...,Beer - Original Organic Lager,Coffee - Irish Cream,Veal - Eye Of Round,Wine - Two Oceans Cabernet,Wine - Chablis 2003 Champs,Bread - French Baquette,Wine - Two Oceans Cabernet,Anchovy Paste - 56 G Tube,Bread - French Baquette,Wine - Two Oceans Cabernet
